## Extra Credit
• Set-up: Let $S_0$ = \\$41.0, $K = \$40.0$, $T = 1$ year, $σ = 30\%$ per annum, $r = 8%$ per annum, $δ = 0$.

In [2]:
import options as opt
import numpy as np
spot = 41
strike = 40
expiry = 1
vol = .3
rate = .08
div = 0.

• Price both European call and put options with the Black-Scholes model and the European Binomial
model with $n = 200$ time steps.

In [3]:
euro = opt.european_binomial_pricer
# amer = opt.american_binomial_pricer
black_call = opt.black_scholes_call
black_put = opt.black_scholes_put

num = 200
euro_call = euro(spot,strike,expiry,rate,div,vol,num,option='call')
euro_put = euro(spot,strike,expiry,rate,div,vol,num,option='put')
# amer_call = amer(spot,strike,expiry,rate,div,vol,num,option='call')
# amer_put = amer(spot,strike,expiry,rate,div,vol,num,option='put')
bs_call = black_call(spot,strike,expiry,rate,div,vol)
bs_put = black_put(spot,strike,expiry,rate,div,vol)

##### • Now write a function that prices the European call and put via Monte Carlo simulation.
– The solution should use your binomial path simulation to simulate $M = 10000$ simulated paths
through the tree.

– This will give you $M$ different terminal stock prices.

In [4]:
def monte_carlo_binomial(spot,expiry,rate,div,vol,num,m) -> np.ndarray:
    li = []
    h = expiry/num
    u = opt.get_u(rate,h,div,vol)
    d = opt.get_d(rate,h,div,vol)
    for i in range(m):
        path = opt.binomial_path(spot,expiry,u,d,num)
        #print(path[-1])
        li.append(path[-1])
    return np.array(li)
    #return np.array([opt.binomial_path(spot,expiry,rate,div,vol,num)[-1] for i in range(m)])

def monte_carlo_normal(spot,expiry,rate,div,vol,num,m) -> np.ndarray:
    li = []
    for i in range(m):
        path = opt.normal_path(spot,expiry,rate,div,vol,num)
        #print(path[-1])
        li.append(path[-1])
    return np.array(li)
    #return np.array([opt.binomial_path(spot,expiry,rate,div,vol,num)[-1] for i in range(m)])

– Get the corresponding option payoffs using the payoff function.

In [4]:
m = 10**4
binom_stocks = monte_carlo_binomial(spot,expiry,rate,div,vol,num,m)
normal_stocks = monte_carlo_normal(spot,expiry,rate,div,vol,num,m)
binom_call_payoffs = opt.call_payoff(binom_stocks,strike)
binom_put_payoffs = opt.put_payoff(binom_stocks,strike)
normal_call_payoffs = opt.call_payoff(normal_stocks,strike)
normal_put_payoffs = opt.put_payoff(normal_stocks,strike)

– Take an average of the option payoffs with the Numpy method `np.mean`.

– Discount this value to time zero and compare it with the Black-Scholes and European Binomial model prices.

In [5]:
binom_call = binom_call_payoffs.mean()*np.exp(-rate*expiry)
binom_put = binom_put_payoffs.mean()*np.exp(-rate*expiry)
normal_call = normal_call_payoffs.mean()*np.exp(-rate*expiry)
normal_put = normal_put_payoffs.mean()*np.exp(-rate*expiry)

print('\t\tCall\tPut')
print(f'European\t${euro_call:.02f}\t${euro_put:.02f}')
# print(f'American\t${amer_call:.02f}\t${amer_put:.02f}')
print(f'Black-Scholes\t${bs_call:.02f}\t${bs_put:.02f}')
print(f'Binom. Sim.\t${binom_call:.02f}\t${binom_put:.02f}')
print(f'Normal Sim.\t${normal_call:.02f}\t${normal_put:.02f}')

		Call	Put
European	$6.97	$2.89
Black-Scholes	$6.96	$2.89
Binom. Sim.	$8.33	$2.43
Normal Sim.	$7.01	$2.85


– Also calculate the standard error of the simulation with `np.std` and divide by `np.sqrt(M)`.

– Repeat for $M = 25000, 50000, 75000, \text{ and } 100000$.

– Make a table to report the data (both discounted mean and standard errors).

In [6]:
print(f'Standard error for the binomial simulation is {binom_stocks.std()/np.sqrt(m)}')
print(f'Standard error for the normal simulation is {normal_stocks.std()/np.sqrt(m)}')

Standard error for the binomial simulation is 0.14406726791697813
Standard error for the normal simulation is 0.138278397847039


In [7]:
# We're not sure why this doesn't satisfy put-call parity
# The payoff functions haven't changed
# We've followed the instructions exactly
print(opt.parity(spot,strike,expiry,rate))
print(abs(binom_call-binom_put))
print(abs(normal_call-normal_put))

4.075346144534571
5.897855445965042
4.161096793986321


– Repeat for $M = 25000, 50000, 75000, \text{ and } 100000$.

– Make a table to report the data (both discounted mean and standard errors).

In [6]:
# Here you can run both binomial and normally distributed move factors to compare
ms = [10000,25000,50000,75000,100000]
print('M\tCall\tPut\tStd Error')
for m in ms:
    #  stock_prices = monte_carlo_binomial(spot,expiry,rate,div,vol,num,m)
    stock_prices = monte_carlo_normal(spot,expiry,rate,div,vol,num,m)
    print(stock_prices.mean())
    call_payoffs = opt.call_payoff(stock_prices,strike)
    put_payoffs = opt.put_payoff(stock_prices,strike)
    
    call_price = call_payoffs.mean() * np.exp(-rate*expiry)
    put_price = put_payoffs.mean() * np.exp(-rate*expiry)
    
    std_error = stock_prices.std()/np.sqrt(m)
    print(f'{m}\t${call_price:.02f}\t${put_price:.02f}\t{std_error}')

M	Call	Put	Std Error
44.60258598922652
10000	$7.13	$2.88	0.13910062396316902
44.298719407481805
25000	$6.87	$2.90	0.08571327209144537
44.4124268835134
50000	$6.96	$2.89	0.06103422170208306
44.41822655069169
75000	$6.95	$2.87	0.04956876169453896
44.40749919108667
100000	$6.96	$2.89	0.04305000029383649


We haven't been able to figure out why the Monte-Carlo simulation doesn't converge to the other values. The simulation also isn't satisfying put-call parity for some reason which is concerning. There is a paper from the 70s that concludes that Monte-Carlo simulations are biased because of the imperfections in pseudo-random number generators. The other possibility is that there aren't enough intermediate steps (`num`/`nper` is not high enough) because the move factors are discrete they take on values that are too extreme. If this is the case, then by the Central Limit Theorem substantially increasing `num` will smooth this out although the few runs with `num` did not change much.

The `binomial_path` function seems to be biased upwards, for what reason we cannot say. However, the mean stock price seems to be about \\$2.00 higher than that of `normal_path`.

We included another function similar to `binomial_path` called `normal_path` that uses move factors generated with a normally distributed random variable to sidestep this problem. As you can see (simply comment out the function you are not using), this will converge to the prices given by the European Binomial and Black-Scholes models.

In [21]:
expiry = 1
num = 3
rate = .08
div = 0
vol = .3
m = 10

h = expiry/num
u = opt.get_u(rate,h,div,vol)
d = opt.get_d(rate,h,div,vol)

print(opt.future_stocks(spot,u,d,num)) # Stock prices at time = 1 after 3 binomial periods
print(monte_carlo_binomial(spot,expiry,rate,div,vol,num,m)) # Stock prices produced by binomial_path

[74.6781323  52.81404438 37.3512727  26.41565494]
[52.81404438 37.3512727  37.3512727  37.3512727  52.81404438 37.3512727
 52.81404438 52.81404438 52.81404438 37.3512727 ]


As you can see, the stock prices that the `binomial_path` function generates are consistent with the possible stock prices along the binomial tree.

In [20]:
print(euro(spot,strike,expiry,rate,div,vol,num,option='call'))
stock_prices = monte_carlo_binomial(spot,expiry,rate,div,vol,num,m=100000)
call_payoffs = opt.call_payoff(stock_prices,strike)
call_price = call_payoffs.mean() * np.exp(-rate*expiry)
print(call_price)

7.073853261277716
8.432380663700624


And still this method of pricing calls returns values that are much too high